### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
24,39941099,58324,79085,51530,0,0,1027,5,2023-03-16 19:35,2023-03-16 19:35,0,3,2501,3,1
25,39938034,41524,79085,3,0,0,1027,6,2023-03-17 19:27,2023-03-17 19:28,1,0,9668,6,1
26,39943999,90578,79085,51530,0,0,1028,0,2023-03-18 22:00,2023-03-18 22:00,1,0,4181,3,1
27,39940908,94579,79085,3,0,0,1028,1,2023-03-19 19:33,2023-03-19 19:36,0,0,10428,4,1
28,39943377,54558,79085,51530,0,0,1028,2,2023-03-20 19:35,2023-03-20 19:35,2,2,1077,0,1
29,39943484,79472,79085,51530,0,0,1028,3,2023-03-21 22:00,2023-03-21 22:00,4,2,922,4,1
30,39942982,35037,79085,3,0,0,1028,4,2023-03-23 01:29,2023-03-23 01:29,4,0,8676,4,1
31,39951740,79665,79085,1001,2,69,1028,6,2023-03-24 21:59,2023-03-24 21:59,1,3,3079,1,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

8

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,58324,39541,31883,0,4,1,1,0,apc1,7,6,6,39941099
1,79085,30416,38074,1,15,2,1,0,5-4-1a,7,8,8,39941099
0,41524,40978,36790,0,4,0,0,0,5-2-2-1,7,6,7,39938034
1,79085,28990,33178,0,9,0,0,0,5-4-1b,6,7,6,39938034
0,90578,38061,35337,0,10,0,0,0,PontaE,8,8,11,39943999
1,79085,31890,34614,0,5,3,1,0,5-4-1a,6,8,9,39943999
0,94579,35865,33290,0,4,2,1,0,4123,6,7,7,39940908
1,79085,34515,37090,4,8,0,0,0,4-3-3a,6,7,8,39940908
0,54558,44837,31238,0,6,1,1,0,433v1,7,8,8,39943377
1,79085,25209,38808,1,13,2,1,0,4-4-2a,7,8,9,39943377


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

756

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1345

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,90095,0,0,0,0,0,0,0,12,1345
1,79722,0,0,0,0,0,0,0,11,1345
2,121094,0,0,0,0,0,0,0,10,1345
3,48541,0,0,0,0,0,0,0,9,1345
4,79085,0,0,0,0,0,0,0,8,1345
5,80120,0,0,0,0,0,0,0,7,1345
6,23755,0,0,0,0,0,0,0,6,1345
7,89208,0,0,0,0,0,0,0,5,1345
8,121466,0,0,0,0,0,0,0,4,1345
9,78652,0,0,0,0,0,0,0,3,1345


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

3

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()